<a href="https://colab.research.google.com/github/muhammadsameerkhan/LLM-Product-Title-Normalization/blob/master/Coca_Cola_Title_Normalization_Prod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.7/328.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00


In [ ]:
%pip install tqdm

In [ ]:
import csv
from openai import OpenAI
import json
from tqdm import tqdm
api_key = ""

**BrainStorming Approach**

In [ ]:
system_message_content = """
    You are an advanced large language model tasked to follow the user instructions strictly.
    To perform the instructed task, learn the patterns from examples delimited by <<<>>>.
    <<<>>>
    Example 1: User input the title = 'Coca-Cola Zero Sugar Zero Koffeinfri 1,5 l'.
    The approach I used to normalize the input title for example 1 is based on following steps:
    First, I translated the whole title to make sure that I can easily understand each word.
    Second, I used the translated title and looked into the ground_truth_flavors which are enclosed in triple commas ''' and tried to match the best
    possible flavor to make sure that it meets the standard as per my organization.
    Third, I removed and add the words in translated title to exactly match with ground_truth_flavor and kept other words as it was in translated title.
    Fourth, After modifing the title in third step, I realized that the volume of drink doesn't look appropriate for display so I did some modifications.
    Finally, my normalized title became => Coca-Cola Zero Koffeinfri 1.5 l.

    Example 2: User input the title = 'Jag vill ha Coca-Cola på köpet'.
    The approach I used to normalize the input title for example 2 is based on following steps:
    First, I translated the whole title to make sure that I can easily understand each word.
    Second, I used the translated title and try to figure out that is it appropriate for display and found out that it sounds like a sentence which
    is not appropriate for a coca-cola product title display so, I removed the extra words and kept only Coca-cola. As there is no flavor or volume
    otherwise I would keep them as well. The extracted version in this case, was => Coca-Cola
    Third, I used the extracted version from second step and looked into the ground_truth_flavors which are enclosed in triple commas '''
    and tried to match the best possible flavor to make sure that it meets the standard as per my organization.
    Fourth, I modified the extracted version with the matched flavor from third step then my modified version was => Coca-Cola Original Taste
    Finally, I again carefully examined this modified version for display purpose so I realized that it misses volume then I added 33 cl into the
    modified version. But, one thing I always kept in my mind that if the volume was present then I wouldn't add 33 cl in this title.
    In this way, my final normalized version looked like this => Coca-Cola Original Taste 33 cl

    Example 3: User input the title = '1 Nduja och 1 Coca Cola'.
    The approach I used to normalize the input title for example 3 is based on following steps:
    First, I translated the whole title to make sure that I can easily understand each word.
    Second, I used the translated title and try to figure out that is it appropriate for display and found out that it sounds like a deal meal which
    is totally appropriate to display because restaurant could sell the coca-cola in deal, so, I decided to just modify the coca-cola title by keeping
    rest of the words same.
    Third, I used the translated title from first step and proceeded with my decision from second step. To modify the coca-cola title, I looked into the
    ground_truth_flavors which are enclosed in triple commas ''' and tried to match the best possible flavor to make sure that it meets the standard as
    per my organization.
    Fourth, I modified the translated title with the best matched flavor from third step then my modified version was => 1 Nduja and 1 Coca-Cola Original Taste
    Finally, I again carefully examined this modified version for display purpose so I realized that it misses volume then I added 33 cl into the
    modified version. But, one thing I always kept in my mind that if the volume was present then I wouldn't add 33 cl in this title.
    In this way, my final normalized version looked like this => 1 Nduja and 1 Coca-Cola Original Taste 33 cl

    Example 4: User input the title = '1 Margherita + 1 Coca Cola/Zero'.
    The approach I used to normalize the input title for example 4 is based on following steps:
    First, I translated the whole title to make sure that I can easily understand each word.
    Second, I used the translated title and try to figure out that is it appropriate for display and found out that it sounds like a deal meal which
    is totally appropriate to display because restaurant could sell the coca-cola in deal. I realized that the Coca-cola in this case have forward
    slash '/' which means that two versions of coca-cola are available in this deal, so, I decided to modify the both versions of coca-cola by keeping
    rest of the words same.
    Third, I used the translated title from first step and proceeded with my decision from second step. To modify the both versions of coca-cola, I
    looked into the ground_truth_flavors which are enclosed in triple commas ''' and tried to match the best possible flavors with each version to make
    sure that it meets the standard as per my organization.
    Fourth, I modified the translated title with the best matched flavors from third step then my modified version
    was => 1 Margherita + 1 Coca-Cola Original Taste/ Coca-Cola Zero
    Finally, I again carefully examined this modified version for display purpose so I realized that infact there are two versions of coca-cola but there should
    be volume present to clearly show the customer so, I added 33 cl into the modified version.  But, one thing I always kept in my mind that if the
    volume was present then I wouldn't add 33 cl in this title.
    In this way, my final normalized version looked like this => 1 Margherita + 1 Coca-Cola Original Taste/ Coca-Cola Zero 33 cl

    Example 5: User input the title = 'Coca-Cola glasflaska 0,5 liters'.
    The approach I used to normalize the input title for example 5 is based on following steps:
    First, I translated the whole title to make sure that I can easily understand each word.
    Second, I used the translated title and try to figure out that is it appropriate for display and found out that it sounds like a title with some
    unnecassary words which don't make sense in display as its obvious, so, I decided to remove the unnecessary words from the translated title. In
    this case the title after removing extra words looked like this 'Coca-Cola 0,5 liters'.
    Third, I used the modified version from second step and looked into the ground_truth_flavors which are enclosed in triple commas ''' and tried
    to match the best possible flavor to make sure that it meets the standard as per my organization as there is no flavor present in the modified version
    so I would choose the 'Coca-Cola Original Taste' from ground_truth_flavors.
    Fourth, I removed and add the words in modified version to exactly match with ground_truth_flavor and kept other words same as it was in modified version.
    Finally, I carefully examined at my modified version from fourth step and realized that the volume of the drink doesn't look appropriate so I decided to convert it,
    my final normalized title became => Coca-Cola Original Taste 50 cl.

    <<<>>>

    '''
    Ground_Truth_Flavors:
    {ground_truth}
    '''

    NOTE: Modify your normalized title by observing patterns from the following provided examples of user input titles on left side and their final normalized title on right side separated by '---'.
    a. Coca-Cola / Pepsi --- Coca-Cola Original Taste/ Pepsi 33 cl (In this example, the volume of drink was missing and there was no explicit word which could be interpereted as volume, therefore we added base volume which is 33cl)
    b. Coca-Cola Original Taste 1,5 liter --- Coca-Cola Original Taste 1.5L
    c. Coca cola sockerfri 0,5 liter --- Coca-Cola Zero 50 cl
    d. Coca-Cola Zero Sugar 1.5L --- Coca-Cola Zero 1.5L
    e. Stor Coca Cola Zero --- Coca-Cola Zero 2L (In this example, Stor means Large so we interpret Large 2L volume of coca-cola)
    f. Medium Coca Cola Zero --- Coca-Cola Zero 50 cl (In this example, medium is interpreted as 50cl)
    g. Coca-Cola Zero Sugar 0kr --- Coca-Cola Zero 33cl (In this example, removed 0kr as it is unnecessary word and the volume was missing so added base volume which is 33cl)
    h. Choose between Coca Cola/Zero/Sprite --- Coca Cola Original Taste/ Coca-Cola Zero/ Sprite (In this example, removed the extra words between and choose because by slashes(/) consumer can easily know it choosable)
    i. Coca-Cola 1/5 l --- Coca-Cola Original Taste 1.5L
    j. Coca Cola --- Coca-Cola Original Taste 33cl (In this example, there is no specific flavor given so I proceeded with base flavor which is 'Original Taste' then the volume of drink was missing and there was no explicit word which could be interpereted as volume, therefore we added base volume which is 33cl)
    k. 70 Deluxe 4 Coca-Cola zero 2 Edamame --- 70 Deluxe 4 Coca-Cola Zero 33cl 2 Edamame (In this example, the volume of drink was missing and there was no explicit word which could be interpereted as volume, therefore we added base volume which is 33cl)
    l. Dryck 0,4cl - Coca Cola Zero --- Coca Cola Zero 40 cl (In this example, the dryck keyword was unnecessary and 0,4cl as volume was written in the starting so I adjusted the positions of coca cola title and then its flavor and then its volume.)
    m. Coca Cola Stor 2L --- Coca-Cola Original Taste 2L (In this example, base flavor is choosen as there was no explicit mention of flavor, Stor and 2L both represents the volume of this drink So I decided to proceed with numerical volume 2L, there is no need to write both stor and 2L)

    Expected Output: Your output should be in JSON format: {response_format}
  """

In [ ]:
def brainstorming_approach(api_key, input_title, correct_titles, response_for, user_mes):

    client = OpenAI(api_key=api_key)

    system_message = system_message_content.format(ground_truth = correct_titles, response_format = response_for)

    user_message = user_mes

    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
      ],
      temperature = 0,
      seed = 1234,
      response_format={"type": "json_object"}
    )

    best_match = response.choices[0].message.content.strip()
    return json.loads(best_match)

In [ ]:
# Load possible correct titles from CSV
correct_titles = []
with open('correct_titles.csv', mode='r') as infile:
    reader = csv.DictReader(infile)
    for row in reader:
        correct_titles.append(row['correct_title'])

correct_titles_string = ", ".join(correct_titles)

# Open the output CSV file for writing matched titles
with open('normalized_titles.csv', mode='w') as outfile:
    fieldnames = ['input_title', 'normalized_title']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()

    # norm_obj = []

    with open('input_titles.csv', mode='r') as infile:

        reader = list(csv.DictReader(infile))
        total_rows = len(reader)

        # Initialize tqdm progress bar
        with tqdm(total=total_rows, desc="Processing Rows", unit="row") as pbar:
            for row in reader:
                input_title = row['input_title']
                best_match = brainstorming_approach(api_key, input_title, correct_titles_string,
                                    response_for="{ 'response' : '<<Your normalized title>>' }",
                                    user_mes=f"Normalize the title: {input_title}")

                # norm_text = best_match.get('response')
                # prediction = brainstorming_approach(api_key, norm_text, correct_titles_string,
                #                     response_for="{ 'response' : '<<rate>>' }",
                #                     user_mes=f"How much this title follows your learnt parameters: {norm_text}, rate out of 10")

                # norm_obj.append({ 'input_title' : input_title, 'normalized_title' : norm_text, 'predicted_rate' : prediction_rate.get('rate') })
                writer.writerow({ 'input_title' : input_title, 'normalized_title' : best_match.get('response')})

                # Update the progress bar
                pbar.update(1)

print("Matching completed and results saved to matched_titles.csv.")

Processing Rows: 100%|██████████| 801/801 [29:51<00:00,  2.24s/row]

Matching completed and results saved to matched_titles.csv.


In [ ]:
input_title = 'Coca-Cola Original Taste'
best_match = brainstorming_approach(api_key, input_title, correct_titles_string,
                                    response_for="{ 'response' : '<<Your normalized title>>' }",
                                    user_mes=f"Normalize the title: {input_title}")


print(best_match.get('response'))

ACCURACY MESUREMENT

In [ ]:
accuracy_check_prompt = """
  You are a detailed oriented large language model tasked to compare generated titles with ground truth labels.
  The generated title: {generated_title}.
  The ground truth label: {ground_truth}.

  Expected Output: Your output should be in JSON format: {response_format}
"""

In [ ]:
def testing(api_key, correct_title, generated_output , user_mes):

    client = OpenAI(api_key=api_key)

    system_message = accuracy_check_prompt.format(ground_truth = correct_title, generated_title = generated_output, response_format = "{ 'accuracy' : 'your_calculated_accuracy' }")

    user_message = user_mes

    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
      ],
      temperature = 0,
      seed = 1234,
      response_format={"type": "json_object"}
    )

    best_match = response.choices[0].message.content.strip()
    return json.loads(best_match)

In [ ]:
usermessage = 'Compare the <generated title> with <ground truth lable> and calculate the accuracy percentage from 100%.'
generated_title = 'Coca-Cola Zero 33 cl'
ground_truth = 'Coca-Cola 33 CL'
testing(api_key, ground_truth, generated_title, usermessage)

{'accuracy': '80%'}